In [ ]:
"""
基于 Graph API 的路由工作流实现
"""

import json
from typing import TypedDict

from langchain_core.messages import HumanMessage, SystemMessage
from langchain_openai import ChatOpenAI
from langgraph.graph import StateGraph, START, END


from dotenv import load_dotenv

load_dotenv()

llm = ChatOpenAI(model="Qwen/Qwen2.5-7B-Instruct")


class State(TypedDict):
    input: str
    decision: str
    output: str


def llm_call_1(state: State):
    """写一个故事"""
    result = llm.invoke(state["input"])
    return {"output": result.content}


def llm_call_2(state: State):
    """写一个笑话"""
    result = llm.invoke(state["input"])
    return {"output": result.content}


def llm_call_3(state: State):
    """写一首诗歌"""
    result = llm.invoke(state["input"])
    return {"output": result.content}


def llm_call_router(state: State):
    model = ChatOpenAI(
        model="Qwen/Qwen2.5-7B-Instruct",
        model_kwargs={"response_format": {"type": "json_object"}},
    )
    ai_msg = model.invoke(
        [
            SystemMessage(
                content="You are a router that directs user input to the appropriate handler. Return a JSON object with a 'step' key and one of the these values: 'story', 'joke', 'poem'. For example: {'step': 'joke'}"
            ),
            HumanMessage(content=state["input"]),
        ]
    )
    decision = json.loads(ai_msg.content)
    return {"decision": decision["step"]}


def route_decision(state: State):
    if state["decision"] == "story":
        return "llm_call_1"
    if state["decision"] == "joke":
        return "llm_call_2"
    if state["decision"] == "poem":
        return "llm_call_3"


router_builder = StateGraph(State)

router_builder.add_node("llm_call_1", llm_call_1)
router_builder.add_node("llm_call_2", llm_call_2)
router_builder.add_node("llm_call_3", llm_call_3)
router_builder.add_node("llm_call_router", llm_call_router)

router_builder.add_edge(START, "llm_call_router")
router_builder.add_conditional_edges(
    "llm_call_router",
    route_decision,
    {
        "llm_call_1": "llm_call_1",
        "llm_call_2": "llm_call_2",
        "llm_call_3": "llm_call_3",
    },
)

router_builder.add_edge("llm_call_1", END)
router_builder.add_edge("llm_call_2", END)
router_builder.add_edge("llm_call_3", END)

router_workflow = router_builder.compile()

state = router_workflow.invoke({"input": "给我写一个关于猫的笑话"})
print(state["output"])

llm_call_2
当然可以！这里有一个关于猫的笑话：

为什么猫喜欢电脑键盘？

因为那里有它们最爱的“回车”键——它们每次都按不动！
